In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [2]:
# Loads IMDb dataset (pre-tokenized as integers)
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

print(f"Training samples: {len(x_train)}, Test samples: {len(x_test)}")

Training samples: 25000, Test samples: 25000


In [3]:
# Ensure all reviews are the same length
maxlen = 200  # truncate/pad to 200 words

x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [4]:
model = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=32, input_length=maxlen),
    layers.SimpleRNN(64),
    layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# Explicitly build the model (for summary to show parameters)
model.build(input_shape=(None, maxlen))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 326,273 (1.24 MB)

 Trainable params: 326,273 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=64,
                    validation_split=0.2)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 27s 79ms/step - accuracy: 0.5335 - loss: 0.6827 - val_accuracy: 0.7766 - val_loss: 0.4832
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 71ms/step - accuracy: 0.7732 - loss: 0.4749 - val_accuracy: 0.8050 - val_loss: 0.4812
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 74ms/step - accuracy: 0.8299 - loss: 0.3967 - val_accuracy: 0.8236 - val_loss: 0.4017
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 73ms/step - accuracy: 0.8423 - loss: 0.3527 - val_accuracy: 0.6466 - val_loss: 0.6086
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 72ms/step - accuracy: 0.8320 - loss: 0.3918 - val_accuracy: 0.7118 - val_loss: 0.5795


In [6]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.7134 - loss: 0.5786
Test Accuracy: 0.7135


In [7]:
# Load the word index mapping used by Keras IMDb dataset
word_index = keras.datasets.imdb.get_word_index()

# Reserve first indices (as per Keras's convention)
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index[""] = 0
word_index[""] = 1
word_index[""] = 2
word_index[""] = 3

# Create reverse mapping (integers to words) if needed
reverse_word_index = {value: key for (key, value) in word_index.items()}

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [8]:
def encode_review(text):
    tokens = text.lower().split()
    encoded = [1]  # Start token
    for word in tokens:
        index = word_index.get(word, 2)  # Use 2 for
        encoded.append(index)
    return keras.preprocessing.sequence.pad_sequences([encoded], maxlen=200)

In [9]:
def predict_sentiment(text):
    encoded = encode_review(text)
    prediction = model.predict(encoded)[0][0]
    label = "Positive 😊" if prediction > 0.5 else "Negative 😞"
    print(f"Review: {text}")
    print(f"Prediction: {prediction:.4f} — {label}")

In [14]:
predict_sentiment("The movie was absolutely fantastic and heartwarming!")
predict_sentiment("It was boring, slow, and I fell asleep halfway through.")
predict_sentiment("This movie was worth watching and I would definitely recommend it.")
predict_sentiment("This movie was a waste of time and money.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Review: The movie was absolutely fantastic and heartwarming!
Prediction: 0.6422 — Positive 😊
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Review: It was boring, slow, and I fell asleep halfway through.
Prediction: 0.2904 — Negative 😞
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Review: This movie was worth watching and I would definitely recommend it.
Prediction: 0.9298 — Positive 😊
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Review: This movie was a waste of time and money.
Prediction: 0.1331 — Negative 😞


In [15]:
model.save('sentiment_rnn_model.keras')  # Recommended format

In [16]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 535.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00


In [17]:
!wget -q -O - ipv4.icanhazip.com

34.80.164.76


In [23]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [31]:
from google.colab import files
uploaded = files.upload()


Saving app.py to app.py


In [38]:
model = keras.models.load_model('sentiment_rnn_model.keras')


In [39]:
model.save('sentiment_rnn_model.keras')


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


⠙

⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.164.76:8501

⠦⠧⠇⠏⠋your url is: https://spotty-days-wonder.loca.lt
